In [1]:
import keras
from keras.layers import Input, Lambda, Dense, Flatten,Dropout
from keras.models import Model,Sequential
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
from glob import glob


Using TensorFlow backend.


In [2]:
!pip install kaggle


In [18]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"haritshandilya","key":"a3b3fc3619ea2aa21d8a587560e99b05"}'}

In [19]:
#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [20]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [21]:
from zipfile import ZipFile
file_name="chest-xray-pneumonia.zip"

with ZipFile (file_name,"r") as zip:
  zip.extractall()
  print("Done!")


Done!


In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/chest_xray/train'
test_path = '/content/chest_xray/test'

In [3]:

# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

model = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [4]:
# don't train existing weights
for layer in model.layers:
    layer.trainable = False

In [5]:
# useful for getting number of output classes
folders = glob('/content/chest_xray/train/*')

In [6]:
x = Flatten()(model.output)

prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=model.input, outputs=prediction)


fc1 = model.layers[-3]
fc2 = model.layers[-2]
predictions = model.layers[-1]

# Create the dropout layers
dropout1 = Dropout(0.75)
dropout2 = Dropout(0.75)

# Reconnect the layers
x = dropout1(fc1.output)
x = fc2(x)
x = dropout2(x)
predictors = predictions(x)

# Create a new model
model = Model(input=model.input, output=predictors)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [7]:
model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [8]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [9]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [11]:
test_set = test_datagen.flow_from_directory('/content/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [12]:
# fitting the model
hist = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
163/163 [==============================] - 111s 679ms/step - loss: 0.6391 - accuracy: 0.8240 - val_loss: 0.4530 - val_accuracy: 0.8718
Epoch 2/5
163/163 [==============================] - 105s 646ms/step - loss: 0.4420 - accuracy: 0.8869 - val_loss: 1.0243 - val_accuracy: 0.8894
Epoch 3/5
163/163 [==============================] - 105s 646ms/step - loss: 0.4600 - accuracy: 0.8932 - val_loss: 0.0965 - val_accuracy: 0.8654
Epoch 4/5
163/163 [==============================] - 105s 645ms/step - loss: 0.5285 - accuracy: 0.8882 - val_loss: 0.6855 - val_accuracy: 0.9054
Epoch 5/5
163/163 [==============================] - 106s 648ms/step - loss: 0.5130 - accuracy: 0.9022 - val_loss: 0.4466 - val_accuracy: 0.9038


In [ ]:
model.save('model_vgg16.h5')

In [15]:
def get_predictions(new_image):
  new_image=image.load_img(new_image,target_size=(224,224))
  new_image=image.img_to_array(new_image)
  new_image=np.expand_dims(new_image,axis=0)
  new_image= preprocess_input(new_image)
  pred =model.predict(new_image)
  final_pred=np.argmax(pred)
  if final_pred==0:
    print("Not Infected - Normal")
  else :
    print("Infected By Pneumonia")



In [16]:
training_set.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [26]:
#Great! Our model is working fine.

get_predictions("/content/chest_xray/val/NORMAL/NORMAL2-IM-1438-0001.jpeg") 

Not Infected - Normal


In [59]:
get_predictions("/content/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg")

Infected By Pneumonia
